# Day 10: Syntax Scoring

In [1]:
import pytest
import dataclasses

In [2]:
def load_data(filename):
    with open(filename) as f:
        for line in f.readlines():
            yield line.strip()

In [3]:
lines = list(load_data('10-sample.txt'))
assert lines[0] == '[({(<(())[]>[[{[]{<()<>>'

In [4]:
OPEN_SYMBOLS = "([{<"
CLOSE_SYMBOLS = ")]}>"

def match(open_symbol, close_symbol):
    assert open_symbol in OPEN_SYMBOLS
    assert close_symbol in CLOSE_SYMBOLS
    return OPEN_SYMBOLS.index(open_symbol) == CLOSE_SYMBOLS.index(close_symbol)

assert match('(', ')') is True
assert match('[', ']') is True
assert match('{', '}') is True
assert match('<', '>') is True
assert match('(', ']') is False
assert match('(', '}') is False
assert match('(', '>') is False
assert match('[', '>') is False
with pytest.raises(AssertionError):
    match('(', 'A')

In [5]:
def expected_for(c):
    assert c in OPEN_SYMBOLS
    index = OPEN_SYMBOLS.index(c)
    return CLOSE_SYMBOLS[index]

assert expected_for('(') == ')'
assert expected_for('[') == ']'
assert expected_for('{') == '}'
assert expected_for('<') == '>'

In [6]:
@dataclasses.dataclass
class Result:
    expected: str
    found: str
        
    def __bool__(self):
        return self.expected == self.found
    
    def __str__(self):
        if self.expected != self.found:
            return f"Expected {self.expected}, but found {self.found} instead."
        return ""
    
    def is_corrupted(self):
        return self.found != ''
    
    
r = Result(expected="}", found=">")
assert bool(r) is False
assert str(r) == 'Expected }, but found > instead.'

r = Result(expected="}", found="}")
assert bool(r) is True
assert str(r) == ''

In [7]:
def check_line(line):
    stack = []
    for c in line:
        if c in OPEN_SYMBOLS:
            stack.append(c)
        elif c in CLOSE_SYMBOLS:
            top = stack.pop()
            if not match(top, c):
                return Result(expected=expected_for(top), found=c)
    if stack:
        return Result(expected=[expected_for(c) for c in stack], found='')
    return Result(expected='', found='')

So, `()` is a legal chunk that contains no other chunks, as is `[]`. More complex but valid chunks include `([])`, `{()()()}`, `<([{}])>`, `[<>({}){}[([])<>]]`, and even `(((((((((())))))))))`.

In [8]:
assert check_line("()")
assert check_line("([])")
assert check_line("{()()()}")
assert check_line("<([{}])>")
assert check_line("[<>({}){}[([])<>]]")
assert check_line("(((((((((())))))))))")

A corrupted line is one where a chunk closes with the wrong character - that is, where the characters it opens and closes with do not form one of the four legal pairs listed above.

Examples of corrupted chunks include `(]`, `{()()()>`, `(((()))}`, and `<([]){()}[{}])`. Such a chunk can appear anywhere within a line, and its presence causes the whole line to be considered corrupted.

In [9]:
assert not check_line("(]")
assert not check_line("{()()()>")
assert not check_line("(((()))}")
assert not check_line("<([]){()}[{}])")

Some of the lines aren't corrupted, just incomplete; you can ignore these lines for now. The remaining five lines are corrupted:

```
{([(<{}[<>[]}>{[]{[(<()> - Expected ], but found } instead.
[[<[([]))<([[{}[[()]]] - Expected ], but found ) instead.
[{[{({}]{}}([{[{{{}}([] - Expected ), but found ] instead.
[<(<(<(<{}))><([]([]() - Expected >, but found ) instead.
<{([([[(<>()){}]>(<<{{ - Expected ], but found > instead.
```

In [10]:
for line in load_data('10-sample.txt'):
    chk = check_line(line)
    if not chk and chk.is_corrupted():
        print(line, chk, chk.found)

{([(<{}[<>[]}>{[]{[(<()> Expected ], but found } instead. }
[[<[([]))<([[{}[[()]]] Expected ], but found ) instead. )
[{[{({}]{}}([{[{{{}}([] Expected ), but found ] instead. ]
[<(<(<(<{}))><([]([]() Expected >, but found ) instead. )
<{([([[(<>()){}]>(<<{{ Expected ], but found > instead. >


Did you know that syntax checkers actually have contests to see who can get the high score for syntax errors in a file? It's true! To calculate the syntax error score for a line, take the **first illegal character on the line** and look it up in the following table:

| char | points        |
|------|---------------|
| `)`  | 3 points      |
| `]`  | 57 points     |
| `}`  | 1197 points   |
| `>`  | 25137 points  |

In the above example, an illegal `)` was found twice (2*3 = 6 points), an illegal `]` was found once (57 points), an illegal `}` was found once (1197 points), and an illegal `>` was found once (25137 points). So, the total syntax error score for this file is:

$$ 6+57+1197+25137 = 26397 $$

**26397** points!


In [11]:
def score(lines):
    points = {
        ")": 3,
        "]": 57,
        "}": 1197,
        ">": 25137,
    }
    result = 0
    for line in lines:
        chk = check_line(line)
        if not chk and chk.is_corrupted():
            result += points[chk.found]
    return result
            
assert score(load_data('10-sample.txt')) ==  26397           

## Solution for part one

In [12]:
sol = score(load_data('10-input.txt')) 
print(f"Solution of part one is: {sol}")

Solution of part one is: 318099


## Part two

You can only use closing characters (), ], }, or >), and you must add them in the correct order so that only legal pairs are formed and all chunks end up closed.

In the example above, there are five incomplete lines:

- `[({(<(())[]>[[{[]{<()<>>` - Complete by adding `}}]])})]`.
- `[(()[<>])]({[<{<<[]>>(` - Complete by adding `)}>]})`.
- `(((({<>}<{<{<>}{[]{[]{}` - Complete by adding `}}>}>))))`.
- `{<[[]]>}<{[{[{[]{()[[[]` - Complete by adding `]]}}]}]}>`.
- `<{([{{}}[<[[[<>{}]]]>[]]` - Complete by adding `])}>`.

In [13]:
for line in load_data('10-sample.txt'):
    chk = check_line(line)
    if not chk.is_corrupted():
        print(line, bool(chk), ''.join(reversed(chk.expected)))

[({(<(())[]>[[{[]{<()<>> False }}]])})]
[(()[<>])]({[<{<<[]>>( False )}>]})
(((({<>}<{<{<>}{[]{[]{} False }}>}>))))
{<[[]]>}<{[{[{[]{()[[[] False ]]}}]}]}>
<{([{{}}[<[[[<>{}]]]>[]] False ])}>


Did you know that autocomplete tools also have contests? It's true! The score is determined by considering the completion string character-by-character. Start with a total score of 0. Then, for each character, multiply the total score by 5 and then increase the total score by the point value given for the character in the following table:

| symbol | points    |
|--------|-----------|
| `)`    | 1 point   |
| `]`    | 2 points  |
| `}`    | 3 points  |
| `>`    | 4 points  |


In [14]:
def autocomplete_score(chars, tron=False):
    points = {
        ")": 1,
        "]": 2,
        "}": 3,
        ">": 4,
    }
    acc = 0
    for c in chars:
        if tron: print(acc, end=" -> ")
        acc *= 5
        acc += points[c]
        if tron: print(acc)
    return acc
    

So, the last completion string above - `])}>` - would be scored as follows:

- Start with a total score of $0$.
- Multiply the total score by $5$ to get $0$, then add the value of ] (2) to get a new total score of $2$.
- Multiply the total score by $5$ to get $10$, then add the value of ) (1) to get a new total score of $11$.
- Multiply the total score by $5$ to get $55$, then add the value of } (3) to get a new total score of $58$.
- Multiply the total score by $5$ to get $290$, then add the value of > (4) to get a new total score of $294$.

In [15]:
assert autocomplete_score('])}>', tron=True) == 294

0 -> 2
2 -> 11
11 -> 58
58 -> 294


The five lines' completion strings have total scores as follows:

- `}}]])})]` - 288957 total points.
- `)}>]})` - 5566 total points.
- `}}>}>))))` - 1480781 total points.
- `]]}}]}]}>` - 995444 total points.
- `])}>` - 294 total points.

In [16]:
for line in load_data('10-sample.txt'):
    chk = check_line(line)
    if not chk.is_corrupted():
        print(line, bool(chk), autocomplete_score(reversed(chk.expected)))

[({(<(())[]>[[{[]{<()<>> False 288957
[(()[<>])]({[<{<<[]>>( False 5566
(((({<>}<{<{<>}{[]{[]{} False 1480781
{<[[]]>}<{[{[{[]{()[[[] False 995444
<{([{{}}[<[[[<>{}]]]>[]] False 294


### Solution part two

Autocomplete tools are an odd bunch: the winner is found by sorting all of the scores and then taking the middle score. (There will always be an odd number of scores to consider.) In this example, the middle score is $288957$ because there are the same number of scores smaller and larger than it.

Find the completion string for each incomplete line, score the completion strings, and sort the scores. What is the middle score?

In [17]:
items = []
for line in load_data('10-sample.txt'):
    chk = check_line(line)
    if not chk.is_corrupted():
        score = autocomplete_score(reversed(chk.expected))
        items.append(score)
items.sort()  
sol = items[len(items)//2]
assert sol == 288957

In [18]:
items = []
for line in load_data('10-input.txt'):
    chk = check_line(line)
    if not chk.is_corrupted():
        score = autocomplete_score(reversed(chk.expected))
        items.append(score)
items.sort()  
sol = items[len(items)//2]
print(f"Solution part two: {sol}")

Solution part two: 2389738699
